

# **Assignment: Training neural network: Learning rate, dropout, activation function**

Trong bài thực hành này, chúng ta sẽ tìm hiểu các vấn đề về learning rate, dropout, activation function thông qua bài toán phân loại chữ số viết tay trên bộ dữ liệu MNIST.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import cv2

import torch
import os
from torch import nn
import torch.nn.functional as F 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard

## **Phần 1: Quan sát dữ liệu**
MNIST là tập dữ liệu ảnh đen trắng các chữ số viết tay, có cùng kích thước 28x28. Chữ số trong ảnh đã được căn chỉnh vào tâm. Đây là tập dữ liệu rất phù hợp cho việc thử nghiệm các kỹ thuật huấn luyện và nhận dạng mẫu mà không đòi hỏi quá nhiều công sức tiền xử lý.

Bộ dữ liệu MNIST được chia sẵn thành 2 phần: tập dữ liệu huấn luyện gồm 60.000 ảnh, tập dữ liệu kiểm thử gồm 10.000 ảnh. Các ảnh trong bộ dữ liệu thuộc về một trong 10 lớp: 0, 1, 2,..., 9.

Thư viện PyTorch đã cung cấp sẵn một module để tải về MNIST:

In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    download = True        
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False
)
(x_train, y_train) = train_data.data[:].detach().numpy(), train_data.targets[:].detach().numpy()
(x_test, y_test) = test_data.data[:].detach().numpy(), test_data.targets[:].detach().numpy()
print('Training image: ', x_train.shape)
print('Testing image: ', x_test.shape)
print('Training label: ', y_train.shape)
print('Testing label: ', y_test.shape)


Để dễ hình dung về dữ liệu, có thể sử dụng thư viện matplotlib quan sát một vài mẫu dữ liệu:

In [ ]:
for i in range(30):
    idx = np.random.randint(0, x_train.shape[0])
    image = x_train[idx]
    plt.subplot(3, 10, i + 1), plt.imshow(image, cmap='gray')
    plt.title(y_train[idx]), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
batch_size = 64
num_classes = 10
epochs = 20
# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

Trước khi đưa vào mô hình, cần chuẩn hoá giá trị điểm ảnh về trong khoảng [0,1] nhằm giúp các thuật toán tối ưu hội tụ nhanh hơn:

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
print(x_train.shape, y_train.shape)

## Phần 2: Xây dựng mô hình phân loại

Trong phần này, chúng ta sẽ xây dựng một mô hình phân loại đơn giản cho bài toán nhận diện chữ số viết tay sử dụng thư viện keras. Từ mô hình này ta sẽ thử nghiệm tác động của các yếu tố như learning rate, dropout, activation function đến quá trình huấn luyện mạng. 

In [ ]:
class DeepModel(nn.Module):
    def __init__(self, dropout_rate):
        super(DeepModel, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, kernel_size=3, stride=(1, 1), padding=0, bias=False, dilation=1)
        self.conv_2 = nn.Conv2d(32, 64, kernel_size=3, stride=(1, 1), padding=0, bias=False, dilation=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(dropout_rate)

        self.dense_1 = nn.Linear(12*12*64, 512)
        self.dense_2 = nn.Linear(512, 10)
    
    def forward(self, x):
        
        x = x.permute(None, None, None, None) # reshape input to (batch, #channels, height, width)
        ### START CODE HERE ~ 5-9 lines
        ## Network Architecture: 2x(Conv2d -> ReLU) -> MaxPool2d -> Flatten -> Linear -> ReLU -> Dropout -> Linear -> Log Softmax (implemented)
        
        ### END CODE HERE
        output = F.log_softmax(x)
        return output

#### Kiểm tra số lượng tham số

In [ ]:
simple_model = DeepModel(dropout_rate = 0.5).cuda()
for param in simple_model.parameters():
    if param.requires_grad:
        print('param autograd')
        break

input = torch.rand(2, 28, 28, 1).cuda()
output = simple_model(input)  # type: torch.Tensor

model_parameters = filter(lambda p: p.requires_grad, simple_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Number of parameter:', params)

assert params==4742954, "Kiểm tra lại phần forward"

####  Khởi tạo Generator

In [ ]:
class Generator(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [ ]:
training_data = Generator(x_train, y_train)
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)

In [ ]:
test_data = Generator(x_test, y_test)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
# Iterate through the Dataloader
data, label = next(iter(train_dataloader))
print(f"Feature batch shape: {data.size()}")
print(f"Labels batch shape: {label.size()}")

# Plot
img = data[0].squeeze()
label = label[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

### Phần 2.1: Vai trò của Dropout 

Kỹ thuật dropout tắt đi một số kết nối một cách ngẫu nhiên trong mỗi lượt huấn luyện, giúp tránh hiện tượng đồng thích nghi (co-adaptation). Điều đó giúp mô hình bị quá khớp. Việc quá khớp thể hiện ở việc lỗi huấn luyện là rất nhỏ nhưng lỗi kiểm thử lại lớn. Phần thực hành tiếp theo nhằm minh hoạ cho tác dụng của kỹ thuật này.

Nếu 1 lớp fully connected có quá nhiều tham số và chiếm hầu hết tham số, các nút mạng trong lớp đó quá phụ thuộc lẫn nhau trong quá trình huấn luyện thì sẽ hạn chế sức mạnh của mỗi nút, dẫn đến việc kết hợp quá mức.

![dropout](https://images.viblo.asia/b621cd15-1862-465e-b11e-6655f3f855c0.png)

Tham khảo: https://towardsdatascience.com/introduction-to-dropout-to-regularize-deep-neural-network-8e9d6b1d4386

#### Dropout rate = 0

In [ ]:
use_cuda = torch.cuda.is_available()  #GPU cuda
best_loss = float('inf')

model = DeepModel(dropout_rate = 0)

optimizer = torch.optim.Adam(model.parameters())
if use_cuda:
    model = torch.nn.parallel.DataParallel(model.cuda())   # , device_ids=[0, 1, 2, 3]
    torch.backends.cudnn.benchmark = True

In [ ]:
def train(model, epoch, writer):
    print('\n ############################# Train phase, Epoch: {} #############################'.format(epoch))
    model.train()
    train_loss = 0
    running_loss = 0
    print('\nLearning rate at this epoch is: ', optimizer.param_groups[0]['lr'], '\n')
    for (batch_idx, target_tuple) in enumerate(train_dataloader):
        if use_cuda:
            target_tuple = [target_tensor.cuda(non_blocking=True) for target_tensor in target_tuple]

        images, labels = target_tuple
        # Convert label to long type pytorch
        labels = torch.tensor(labels,dtype=torch.long)

        optimizer.zero_grad()  # zero the gradient buff
        output_tuple = model(images)

        loss = F.nll_loss(output_tuple, labels).cuda()

        loss.backward()  # retain_graph=True
        optimizer.step()

        train_loss += loss.item()  # loss　　　
        running_loss += loss.item()
        if batch_idx % 50 == 49:
            writer.add_scalar('training loss', running_loss/50, epoch * len(train_dataloader) + batch_idx)
            running_loss = 0
        #print('########################### Epoch:', epoch, ', --  batch:',  batch_idx, '/', len(train_dataloader), ',   ',
        #      'Train loss: %.3f, accumulated average loss: %.3f ##############################' % (loss.item(), train_loss / (batch_idx + 1)))



In [ ]:
def test(model, epoch, writer):
    print('\n ############################# Test phase, Epoch: {} #############################'.format(epoch))
    model.eval()
    with torch.no_grad():
        test_loss = 0
        correct = 0
        for (batch_idx, target_tuple) in enumerate(test_dataloader):
            if use_cuda:
                target_tuple = [target_tensor.cuda(non_blocking=True) for target_tensor in target_tuple]

            images, labels = target_tuple
            # Convert label to long type pytorch
            labels = torch.tensor(labels,dtype=torch.long)
            output_tuple = model(images)
            #print(output_tuple.shape)

            _, predicted = torch.max(output_tuple.data, 1)
            correct += (predicted == labels).sum().item()

            loss = F.nll_loss(output_tuple, labels).cuda()

            test_loss += loss.item()  # loss　　　
            #print('########################### Epoch:', epoch, ', --  batch:',  batch_idx, '/', len(test_dataloader), ',   ',
            #     'Test loss: %.3f, accumulated average loss: %.3f ##############################' % (loss.item(), test_loss / (batch_idx + 1)))
        acc = correct*100/len(test_data)
        print('Accuracy: ', acc)
        writer.add_scalar('test accuracy', acc, epoch)



In [ ]:
def train_and_test(model, epoch_num = 5, summary_path='runs/mnist_experiment_dropout'):
    writer = SummaryWriter(summary_path)
    for epoch in range(epoch_num):
        train(model, epoch, writer)
        test(model, epoch, writer)

Huấn luyện mô hình

In [ ]:
train_and_test(model, 5, 'runs/mnist_experiment_dropout=0')

In [ ]:
%tensorboard --logdir=runs

#### Dropout rate = 0.5

In [ ]:
use_cuda = torch.cuda.is_available()  #GPU cuda
best_loss = float('inf')

### START CODE HERE ~ 1 lines
## Create an instance of DeepModel with dropout = 0.5 and load it to cuda()

### END CODE HERE

optimizer = torch.optim.Adam(model.parameters())
# if use_cuda:
#     model = torch.nn.parallel.DataParallel(model.cuda())   # , device_ids=[0, 1, 2, 3]
#     torch.backends.cudnn.benchmark = True

In [ ]:
train_and_test(model, optimizer, 5, 'runs/mnist_experiment_dropout=0.5')

In [ ]:
%tensorboard --logdir=runs

#### **Nhận xét và chú ý**
Nhận xét:
- Dropout sẽ được học thêm các tính năng mạnh mẽ hữu ích
- Nó gần như tăng gấp đôi số epochs cần thiết để hội tụ. Tuy nhiên, thời gian cho mỗi epoch là ít hơn. Trong phần visualization, các bạn có thể thấy mặc dù mặc dù độ chính xác của phần dropout=0.5 thấp hơn lúc không áp dụng dropout nhưng trên trên tập test độ chính xác lại cao hơn, và nó cũng cần nhiều epoch hơn để hội tụ.
- Ta có H đơn vị ẩn, với xác suất bỏ học cho mỗi đơn vị là (1 - p) thì ta có thể có 2^H mô hình có thể có. Nhưng trong giai đoạn test, tất cả các nút mạng phải được xét đến, và mỗi activation sẽ giảm đi 1 hệ số p.

Chú ý:
- Không dùng Dropout cho quá trình test
- Áp dụng Dropout cho cả quá trình Forward và Backward
- Giá trị kích hoạt phải giảm đi 1 hệ số keep_prob, tính cả cho những nút bỏ học.

### Phần 2.2: Vai trò của activation function
Để nói về vai trò của hàm kích hoạt, vậy hãy thử đặt ra câu hỏi: **Chuyện gì sẽ xảy ra nếu không có các hàm kích hoạt (hàm phi tuyến) này?**

Hãy tưởng tượng rằng thay vì áp dụng 1 hàm phi tuyến, ta chỉ áp dụng 1 hàm tuyến tính vào đầu ra của mỗi neuron. Vì phép biến đổi không có tính chất phi tuyến, việc này không khác gì chúng ta thêm một tầng ẩn nữa vì phép biến đổi cũng chỉ đơn thuần là nhân đầu ra với các weights. Với chỉ những phép tính đơn thuần như vậy, trên thực tế mạng neural sẽ không thể phát hiện ra những quan hệ phức tạp của dữ liệu (ví dụ như: dự đoán chứng khoán, các bài toán xử lý ảnh hay các bài toán phát hiện ngữ nghĩa của các câu trong văn bản). Nói cách khác nếu không có các activation functions, khả năng dự đoán của mạng neural sẽ bị giới hạn và giảm đi rất nhiều, sự kết hợp của các activation functions giữa các tầng ẩn là để giúp mô hình học được các quan hệ phi tuyến phức tạp tiềm ẩn trong dữ liệu.

Tham khảo: 
- https://viblo.asia/p/mot-so-ham-kich-hoat-trong-cac-mo-hinh-deep-learning-tai-sao-chung-lai-quan-trong-den-vay-part-1-ham-sigmoid-bWrZn4Rv5xw

- https://towardsdatascience.com/the-importance-and-reasoning-behind-activation-functions-4dc00e74db41#:~:text=Why%20do%20we%20need%20them,a*x%2Bb).

Viết mô hình đơn giản cho phép nhận các loại hàm kích hoạt khác nhau

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, dropout_rate =0.5, activation = None):
        super(SimpleModel, self).__init__()
        self.dense_1 = nn.Linear(28*28, 512)
        self.dropout = nn.Dropout(dropout_rate)
        self.dense_2 = nn.Linear(512, 10)
        self.activation = activation
    
    def forward(self, x):
        ### START CODE HERE ~ 6 lines
        ## Network Architecture: Flatten(input) -> Linear -> activation -> Dropout -> Linear -> Log Softmax
        ## The activation of the first Linear will be passed when creating an instance of model 

        ### END CODER HERE
        return output

Kiểm tra số lượng tham số


In [ ]:
simple_model = SimpleModel().cuda()
for param in simple_model.parameters():
    if param.requires_grad:
        print('param autograd')
        break

input = torch.rand(1, 28, 28).cuda()
output = simple_model(input)  # type: torch.Tensor

model_parameters = filter(lambda p: p.requires_grad, simple_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Number of parameter:', params)

assert params==407050, "Kiểm tra lại phần forward"

Trước tiên hãy thử xem, sẽ thế nào nếu không sử dụng hàm kích hoạt cho lớp ẩn:

In [ ]:
### START CODE HERE
simple_model = None # Do not forget pass model to cuda
optimizer = None # Adam optimizer
### ENCODE CODE HERE
train_and_test(simple_model, optimizer, 5, 'runs/mnist_none_activation')

Kết quả đạt được là tương đối tệ trên tập dữ liệu MNIST. Tiếp theo chúng ta sẽ thử nghiệm và so sánh kết quả khi sử dụng các hàm kích hoạt khác nhau:

In [ ]:
for activation in [None, nn.Sigmoid(), nn.Tanh(), nn.ReLU()]:
    print("Activation: ", str(activation))
    ### START CODE HERE
    simple_model = None # Do not forget to pass model to cuda
    optimizer = None # Adam optimizer
    ### END CODE HERE
    train_and_test(simple_model, optimizer, 5, 'runs/mnist_' + str(activation))

In [ ]:
%tensorboard --logdir=runs

### Phần 2.3: Vai trò của hệ số học learning rate

Ta tiếp tục quan sát tác động của learning rate đến quá trình học của mạng.

Tham khảo: https://viblo.asia/p/learning-rate-nhung-dieu-co-the-ban-da-bo-qua-gGJ59BV9KX2

In [ ]:
learning_rates = [1E-0, 1E-1, 1E-2, 1E-3, 1E-4, 1E-5, 1E-6, 1E-7]
for lr in learning_rates:
    ### START CODE HERE
    print('Learning rate: %f' % lr)
    simple_model = None
    optimizer = None # SGD with momentum = 0.9
    None # train and test these models
    ### END CODE HERE

In [ ]:
%tensorboard --logdir=runs

#### Giảm learning rate theo cơ chế: `learning_rate = init_learning_rate / (1 + decay * step)`

In [ ]:
def train(model, optimizer, scheduler, epoch, writer):
    print('\n ############################# Train phase, Epoch: {} #############################'.format(epoch))
    model.train()
    train_loss = 0
    running_loss = 0
    print('\nLearning rate at this epoch is: ', scheduler.get_last_lr(), '\n')
    for (batch_idx, target_tuple) in enumerate(train_dataloader):
        if use_cuda:
            target_tuple = [target_tensor.cuda(non_blocking=True) for target_tensor in target_tuple]

        images, labels = target_tuple
        # Convert label to long type pytorch
        labels = torch.tensor(labels,dtype=torch.long)

        optimizer.zero_grad()  # zero the gradient buff
        output_tuple = model(images)

        loss = F.nll_loss(output_tuple, labels).cuda()

        loss.backward()  # retain_graph=True
        optimizer.step()
        

        train_loss += loss.item()  # loss　　　
        running_loss += loss.item()
        if batch_idx % 50 == 49:
            writer.add_scalar('training loss', running_loss/50, epoch * len(train_dataloader) + batch_idx)
            running_loss = 0
        #print('########################### Epoch:', epoch, ', --  batch:',  batch_idx, '/', len(train_dataloader), ',   ',
        #      'Train loss: %.3f, accumulated average loss: %.3f ##############################' % (loss.item(), train_loss / (batch_idx + 1)))
    scheduler.step()

In [ ]:
def train_and_test(model, optimizer, scheduler, epoch_num = 5, summary_path='runs/mnist_experiment_dropout'):
    writer = SummaryWriter(summary_path)
    for epoch in range(epoch_num):
        train(model, optimizer, scheduler, epoch, writer)
        test(model, epoch, writer)

Viết đoạn code cho phép huấn luyện mô hình với tốc độ học giảm dần qua từng epoch với hệ số 0.9 sử dụng các schedule learning rate `ExponentialLR`

In [ ]:
### START CODE HERE
simple_model = None # use ReLU, dropout = 0.5
optimizer = None # SGD
scheduler = None # ExponentialLR with gamma=0.9
### END CODE HERE
train_and_test(simple_model, optimizer, scheduler, 20, 'runs/exponentialLR')

In [ ]:
%tensorboard --logdir=runs